# AceleraDev Codenation Semana 3 Aula 5

Serão analisados os dados disponíveis no Kaggle:
`https://www.kaggle.com/rubenssjr/brasilian-houses-to-rent/data`

## Colunas

- id
- city: Cidade onde o imóvel está localizada / City where the property is located
- area: Area do imovel / Property area
- rooms: Numero de quartos/ Quantity of rooms
- bathroom: Numero de banheiros / Quantity of bathroom
- parking spaces: Numero de vagas / Quantity of parking spaces
- floor: Andar / Floor
- animal: Aceita animais? / Acept animals?
- furniture: Mobilhada? / Furniture?
- hoa: Valor do condominio / Homeowners association tax
- rent amount: Valor do Aluguel / Rent amount
- property tax: IPTU / Property tax
- fire insurance:  Seguro Incendio / Fire Insurance
- total: Valor total / Total

## Importando os pacotes

In [ ]:
import pandas as pd

## Realizando a análise inicial do DataFrame

In [ ]:
df = pd.read_csv('houses_to_rent_v2.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

## Problema: Explorar o valor do aluguel (rent amount R$)

### Estatística univariada

In [ ]:
df.rename(columns={'rent amount (R$)': 'valor_aluguel'}, inplace=True)

In [ ]:
df.valor_aluguel.mean()

In [ ]:
df.valor_aluguel.median()

Como a mediana é menor que a média, já podemos prever que os dados não serão uniformemente distribuídos, e que podemos ter outliers no conjunto

In [ ]:
df.valor_aluguel.std()

O valor elevado do desvio padrão indica que os dados são bastante heterogêneos

In [ ]:
df.valor_aluguel.describe()

Podemos ver que do 75% para o valor máximo, existe um crescimento de 9x do valor, indicando presença de outliers

In [ ]:
df.valor_aluguel.plot(kind='hist', bins=100)

O histograma nos mostra que temos uma concentração maior de valores menores de aluguel, com quantidades menores ao passar de 5000.
Isso reforça a informação anterior exibida pelo describe (75% estão entre 0 e 5000)

In [ ]:
# Analisar a assimetria e a curtose não fazem muito sentido com essa distribuição, pois ela é bem diferente de uma distribuição normal, porém, a título de didática
df.valor_aluguel.skew()
# A assimetria > 0 representa que os dados se concentram mais próximos da origem

In [ ]:
df.valor_aluguel.kurtosis()
# A Curtose maior que zero representa uma curva leptocúrtica, ou seja, mais afunilada

### Estatística descritiva multivariada (Verifica antes o conteúdo da aula 07)

Nesta parte estaremos em busca de **perguntas** e **hipóteses**

#### Perguntas
- Qual a cidade com a média de aluguel mais cara?
- Quantos banheiros existem nas residências com aluguéis mais caros?
- Os imóveis mais caros aceitam animal?
- Os imóveis mais caros são mobiliados?



In [ ]:
# Qual a cidade com a média de aluguel mais alta?
df.groupby('city').agg({"valor_aluguel": ["mean","median"]}).sort_values(('valor_aluguel', 'mean'), ascending=False)

In [ ]:
# Quantos banheiros em média existem nas residencias com aluguéis mais altos
df.valor_aluguel.describe()

In [ ]:
# Vamos considerar aluguéis mais altos como os 25% maiores (acima de 5000)
valorAlto = df.valor_aluguel.describe()["75%"]
df[df.valor_aluguel > valorAlto].bathroom.mean().round(0)
# Com isso podemos afirmar que os aluguéis mais caros possuem em média 4 banheiros

In [ ]:
# Os imóveis mais caros aceitam animais?
df[df.valor_aluguel > valorAlto].animal.value_counts().acept/df[df.valor_aluguel > valorAlto].animal.count()
# Com isso podemos ver que 82% dos aluguéis mais caros aceitam animais

In [ ]:
# Os imóveis mais caros são mobiliados?
df[df.valor_aluguel > valorAlto].furniture.value_counts().furnished / df[df.valor_aluguel > valorAlto].furniture.count()
# Assim podemos ver que apenas 35% dos imóveis mais caros são mobiliados

#### Hipóteses
- São Paulo tem o aluguel mais caro
- Quanto mais banheiros em um imóvel maior o valor do aluguel
-  Os imóveis com mobilia tem o aluguel mais alto

In [ ]:
# São Paulo tem o aluguel mais caro
df[['valor_aluguel', 'bathroom']].corr(method='spearman')
# O valor é acima de 0,7 representando uma correlação alta entre as variáveis

In [ ]:
# Exibindo a correlação de todos os valores 'int64' com o valor do aluguel
df.select_dtypes('int64').corr().valor_aluguel.sort_values(ascending=False)

## Visualização de dados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(x = 'city', data = df)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Média do valor de aluguel por cidade")
sns.barplot(x='city', y='valor_aluguel', data = df.groupby('city').agg({'valor_aluguel':'mean'}).reset_index())
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sns.distplot(df['valor_aluguel'])

In [ ]:
sns.scatterplot(x='valor_aluguel', y='bathroom', data=df, hue='city', size='animal')

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
g = sns.FacetGrid(df, col='city', row='animal')
g = g.map(plt.hist, 'valor_aluguel')